<a href="https://colab.research.google.com/github/Yan-Weijia/CompLing/blob/master/HW0402.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install deeppavlov

In [0]:
!pip install tensorflow==1.15.0

In [0]:
!apt-get --yes install git

In [0]:
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

In [0]:
import pandas as pd

In [0]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/pristavki.csv

In [0]:
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

In [0]:
!wget https://raw.githubusercontent.com/Yan-Weijia/CompLing/master/HW04data/test.txt
!wget https://raw.githubusercontent.com/Yan-Weijia/CompLing/master/HW04data/train.txt
!wget https://raw.githubusercontent.com/Yan-Weijia/CompLing/master/HW04data/valid.txt

In [10]:
!ls *.txt

test.txt  train.txt  valid.txt


In [12]:
!head test.txt

Brother B-PRS
In I-PRS
Arms I-PRS
: I-PRS
Earned I-PRS
In I-PRS
Blood I-PRS
- O
продолжение O
отменного O


In [0]:
import json
from deeppavlov import configs, build_model, train_model

In [14]:
with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as f:
    ner_config = json.load(f)

ner_config['dataset_reader']['data_path'] = './'  # directory with train.txt, valid.txt and test.txt files
ner_config['metadata']['variables']['NER_PATH'] = './'
ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]  # do not download the pretrained ontonotes model

ner_model = train_model(ner_config, download=True)

2020-01-27 23:30:21.561 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip to /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip
100%|██████████| 663M/663M [03:36<00:00, 3.06MB/s]
2020-01-27 23:33:57.959 INFO in 'deeppavlov.core.data.utils'['utils'] at line 237: Extracting /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip archive into /root/.deeppavlov/downloads/bert_models
2020-01-27 23:34:04.566 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...


[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.
2020-01-27 23:34:09.926 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /content/tag.dict]


Using TensorFlow backend.





The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use standard 

...

{"valid": {"eval_examples_count": 1, "metrics": {"ner_f1": 7.8431, "ner_token_f1": 30.6818}, "time_spent": "0:00:03"}}


2020-01-27 23:37:17.933 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 236 tokens with 8 phrases; found: 41 phrases; correct: 0.

precision:  0.00%; recall:  0.00%; FB1:  0.00

	PRS: precision:  0.00%; recall:  0.00%; F1:  0.00 41




{"test": {"eval_examples_count": 1, "metrics": {"ner_f1": 0, "ner_token_f1": 19.917}, "time_spent": "0:00:02"}}


2020-01-27 23:37:18.567 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/tag.dict]
2020-01-27 23:37:46.226 INFO in 'deeppavlov.models.bert.bert_sequence_tagger'['bert_sequence_tagger'] at line 251: [initializing model with Bert from /root/.deeppavlov/downloads/bert_models/multi_cased_L-12_H-768_A-12/bert_model.ckpt]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/downloads/bert_models/multi_cased_L-12_H-768_A-12/bert_model.ckpt


In [15]:
ner_model(['Brother In Arms: Earned In Blood', 'Brother In Arms: Road To Hill 30'])

[[['Brother', 'In', 'Arms', ':', 'Earned', 'In', 'Blood'],
  ['Brother', 'In', 'Arms', ':', 'Road', 'To', 'Hill', '30']],
 [['O', 'O', 'O', 'O', 'B-PRS', 'O', 'O'],
  ['O', 'O', 'O', 'O', 'B-PRS', 'B-PRS', 'O', 'B-PRS']]]

In [16]:
ner_model(['fifa 17', 'фифа 15, 16', 'гта 5', 'GTA V', 'видмак 3'])

[[['fifa', '17'], ['гта', '5'], ['GTA', 'V']],
 [['B-PRS', 'O'], ['B-PRS', 'B-PRS'], ['B-PRS', 'B-PRS']]]

In [17]:
ner_model(['Street Fighter IV', 'Grand Theft Auto 4', 'Call of Duty: Black Ops III', 'Gold of Wars'])

[[['Street', 'Fighter', 'IV'],
  ['Grand', 'Theft', 'Auto', '4'],
  ['Call', 'of', 'Duty', ':', 'Black', 'Ops', 'III']],
 [['O', 'O', 'O'],
  ['O', 'B-PRS', 'O', 'O'],
  ['B-PRS', 'B-PRS', 'O', 'O', 'O', 'O', 'O']]]

In [0]:
marked = []

for text in data.text.values[:1000]:
    # BERT имеет лимит на длину текста в 512 слов, возьмем даже еще меньше
    if len(text.split()) > 100:
        continue
    pred = ner_model([text])
    sent, tags = pred[0][0], pred[1][0]
    
    # достанем только тексты с сущностями
    if len(set(tags[0])) > 1:
        marked.append(list(zip(sent,tags)))

In [0]:
marked

[[('1867', 'B-DATE'),
  ('год', 'I-DATE'),
  ('—', 'O'),
  ('исполнение', 'O'),
  ('Andante', 'B-PERSON'),
  ('и', 'O'),
  ('скерцо', 'B-PERSON'),
  ('из', 'O'),
  ('Первой', 'O'),
  ('симфонии', 'B-PERSON'),
  ('в', 'O'),
  ('Русском', 'B-PLACE'),
  ('музыкальном', 'B-PLACE'),
  ('обществе', 'I-PLACE'),
  ('в', 'O'),
  ('Санкт', 'B-PLACE'),
  ('-', 'O'),
  ('Петербурге', 'B-PLACE'),
  ('.', 'O')],
 [('1866', 'I-DATE'),
  ('—', 'O'),
  ('1867', 'B-DATE'),
  ('годы', 'I-DATE'),
  ('—', 'O'),
  ('написаны', 'O'),
  ('увертюра', 'O'),
  ('на', 'O'),
  ('датский', 'O'),
  ('гимн', 'B-PERSON'),
  ('и', 'O'),
  ('ряд', 'O'),
  ('фортепьянных', 'B-PERSON'),
  ('пьес', 'B-PERSON'),
  (':', 'O'),
  ('«', 'I-WORK_OF_ART'),
  ('Воспоминание', 'I-WORK_OF_ART'),
  ('о', 'I-WORK_OF_ART'),
  ('Гапсале', 'I-WORK_OF_ART'),
  ('»', 'I-WORK_OF_ART'),
  ('.', 'O')],
 [('1867', 'B-DATE'),
  ('год', 'I-DATE'),
  ('—', 'O'),
  ('начата', 'O'),
  ('работа', 'O'),
  ('над', 'O'),
  ('оперой', 'B-WORK_OF_ART'),